--- Day 5: Sunny with a Chance of Asteroids ---

You're starting to sweat as the ship makes its way toward Mercury. The Elves suggest that you get the air conditioner working by upgrading your ship computer to support the Thermal Environment Supervision Terminal.

The Thermal Environment Supervision Terminal (TEST) starts by running a diagnostic program (your puzzle input). The TEST diagnostic program will run on your existing Intcode computer after a few modifications:

First, you'll need to add two new instructions:

Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.
Programs that use these instructions will come with documentation that explains what should be connected to the input and output. The program 3,0,4,0,99 outputs whatever it gets as input, then halts.

Second, you'll need to add support for parameter modes:

Each parameter of an instruction is handled based on its parameter mode. Right now, your ship computer already understands parameter mode 0, position mode, which causes the parameter to be interpreted as a position - if the parameter is 50, its value is the value stored at address 50 in memory. Until now, all parameters have been in position mode.

Now, your ship computer will also need to handle parameters in mode 1, immediate mode. In immediate mode, a parameter is interpreted as a value - if the parameter is 50, its value is simply 50.

Parameter modes are stored in the same value as the instruction's opcode. The opcode is a two-digit number based only on the ones and tens digit of the value, that is, the opcode is the rightmost two digits of the first value in an instruction. Parameter modes are single digits, one per parameter, read right-to-left from the opcode: the first parameter's mode is in the hundreds digit, the second parameter's mode is in the thousands digit, the third parameter's mode is in the ten-thousands digit, and so on. Any missing modes are 0.

For example, consider the program 1002,4,3,4,33.

The first instruction, 1002,4,3,4, is a multiply instruction - the rightmost two digits of the first value, 02, indicate opcode 2, multiplication. Then, going right to left, the parameter modes are 0 (hundreds digit), 1 (thousands digit), and 0 (ten-thousands digit, not present and therefore zero):

```

ABCDE
 1002

DE - two-digit opcode,      02 == opcode 2
 C - mode of 1st parameter,  0 == position mode
 B - mode of 2nd parameter,  1 == immediate mode
 A - mode of 3rd parameter,  0 == position mode,
                                  omitted due to being a leading zero

```
This instruction multiplies its first two parameters. The first parameter, 4 in position mode, works like it did before - its value is the value stored at address 4 (33). The second parameter, 3 in immediate mode, simply has value 3. The result of this operation, 33 * 3 = 99, is written according to the third parameter, 4 in position mode, which also works like it did before - 99 is written to address 4.

Parameters that an instruction writes to will never be in immediate mode.


Finally, some notes:

It is important to remember that the instruction pointer should increase by the number of values in the instruction after the instruction finishes. Because of the new instructions, this amount is no longer always 4.
Integers can be negative: 1101,100,-1,4,0 is a valid program (find 100 + -1, store the result in position 4).
The TEST diagnostic program will start by requesting from the user the ID of the system to test by running an input instruction - provide it 1, the ID for the ship's air conditioner unit.

It will then perform a series of diagnostic tests confirming that various parts of the Intcode computer, like parameter modes, function correctly. For each test, it will run an output instruction indicating how far the result of the test was from the expected value, where 0 means the test was successful. Non-zero outputs mean that a function is not working correctly; check the instructions that were run before the output instruction to see which one failed.

Finally, the program will output a diagnostic code and immediately halt. This final output isn't an error; an output followed immediately by a halt means the program finished. If all outputs were zero except the diagnostic code, the diagnostic program ran successfully.

After providing 1 to the only input instruction and passing all the tests, what diagnostic code does the program produce?

In [59]:
pos(p) = p + 1;
ref(a,p) = a[pos(a[pos(p)])]; 

In [60]:
@enum Op plus=1 mul=2 stor=3 out=4 halt=99
@enum Mode posi=0 imm=1

struct Operation 
   op::Op 
   mode1::Mode
   mode2::Mode
   mode3::Mode
end


In [61]:
println(1002 % 100)
println((1002 ÷ 100) % 10)
println((1002 ÷ 1000) % 10)
println((1002 ÷ 10000) % 10)

2
0
1
0


In [62]:
getOps(str::String) = getOps(parse(Int,str))
function getOps(num)  
    
        op = Op(num % 100)
        mode1 = Mode((num ÷ 100)  % 10)
        mode2 = Mode((num ÷ 1000)  % 10)
        mode3 = Mode((num ÷ 10000)  % 10)
        
        Operation(op,mode1,mode2,mode3)
end

getOps (generic function with 2 methods)

In [63]:
getOps(2)

Operation(mul, posi, posi, posi)

In [64]:
getOps(3)

Operation(stor, posi, posi, posi)

In [65]:
getOps(1002)

Operation(mul, posi, imm, posi)

In [66]:
function optype(str::String)
    
end

optype (generic function with 1 method)

Opcode 1 adds together numbers read from two positions and stores the result in a third position. The three integers immediately after the opcode tell you these three positions - the first two indicate the positions from which you should read the input values, and the third indicates the position at which the output should be stored.
For example, if your Intcode computer encounters 1,10,20,30, it should read the values at positions 10 and 20, add those values, and then overwrite the value at position 30 with their sum.

Opcode 2 works exactly like opcode 1, except it multiplies the two inputs instead of adding them. Again, the three integers after the opcode indicate where the inputs and outputs are, not their value

In [67]:
function op1(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    prog[pos(prog[pos(p+3)])] = p1 + p2
    p + 4
end

function op2(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end    
    
    prog[pos(prog[pos(p+3)])] = p1 * p2
    p + 4
end



op2 (generic function with 2 methods)


Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.



In [210]:
function op3(operation,p,prog,input) 
#    println("entering stor ","p ",p," prog ",prog, " input ",input)
    p1 = prog[pos(p+1)]
    #println("storage position",p1)
    prog[pos(p1)] = input
    #println("exiting stor")  
    p + 2
end

op3 (generic function with 2 methods)

Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.

In [211]:
function op4(operation,p,prog) 
    #println("entering out")     
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  

    println("output:",p1)
    #println("exiting out")    
    p + 2
end

op4 (generic function with 1 method)

In [212]:
function execute(op::Operation, p, prog)
    if op.op == plus
        return op1(op,p,prog)
    end
    if op.op == mul
        return op2(op,p,prog)
    end
    if op.op == stor
        return op3(op,p,prog,1)
    end
    if op.op == out
        return op4(op,p,prog)
    end
    error("panic executing unknown op")
end

execute (generic function with 2 methods)

In [213]:
function execute(prog)
    p = 0
    op = getOps(prog[pos(p)])
    #println("op: ", op)
    while op.op != halt
        #println(prog)
        p = execute(op,p,prog)   
        #println("instruction: ",p, " -> ",prog[pos(p)])
        op = getOps(prog[pos(p)])
    end 
    prog
end;

In [214]:
println(execute([1,9,10,3,2,3,11,0,99,30,40,50]) == [3500,9,10,70,2,3,11,0,99,30,40,50])
println(execute([1,0,0,0,99]) == [2,0,0,0,99])
println(execute([2,3,0,3,99]) == [2,3,0,6,99])
println(execute([2,4,4,5,99,0]) == [2,4,4,5,99,9801])
println(execute([1,1,1,4,99,5,6,0,99]) == [ 30,1,1,4,2,5,6,0,99])

true
true
true
true
true


In [215]:
execute([3,0,4,0,99])

output:1


5-element Array{Int64,1}:
  1
  0
  4
  0
 99

In [216]:
using DelimitedFiles
prog = readdlm("input.txt", '\t', Int, ',')
execute(prog);
# expected output is: 13787043

output:0
output:0
output:0
output:0
output:0
output:0
output:0
output:0
output:0
output:13787043


678×1 Array{Int64,2}:
    3
  225
    1
  225
    6
    6
 1101
    1
  238
  225
  104
    0
 1102
    ⋮
  223
 1005
  224
  674
 1001
  223
    1
  223
    4
  223
   99
  226

he air conditioner comes online! Its cold air feels good for a while, but then the TEST alarms start to go off. Since the air conditioner can't vent its heat anywhere but back into the spacecraft, it's actually making the air inside the ship warmer.

Instead, you'll need to use the TEST to extend the thermal radiators. Fortunately, the diagnostic program (your puzzle input) is already equipped for this. Unfortunately, your Intcode computer is not.

Your computer is only missing a few opcodes:

```
Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.

Opcode 6 is jump-if-false: if the first parameter is zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.

Opcode 7 is less than: if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

Opcode 8 is equals: if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

Like all instructions, these instructions need to support parameter modes as described above.
```

Normally, after an instruction is finished, the instruction pointer increases by the number of values in that instruction. However, if the instruction modifies the instruction pointer, that value is used and the instruction pointer is not automatically increased.

For example, here are several programs that take one input, compare it to the value 8, and then produce one output:

```
3,9,8,9,10,9,4,9,99,-1,8 - Using position mode, consider whether the input is equal to 8; output 1 (if it is) or 0 (if it is not).
3,9,7,9,10,9,4,9,99,-1,8 - Using position mode, consider whether the input is less than 8; output 1 (if it is) or 0 (if it is not).
3,3,1108,-1,8,3,4,3,99 - Using immediate mode, consider whether the input is equal to 8; output 1 (if it is) or 0 (if it is not).
3,3,1107,-1,8,3,4,3,99 - Using immediate mode, consider whether the input is less than 8; output 1 (if it is) or 0 (if it is not).
Here are some jump tests that take an input, then output 0 if the input was zero or 1 if the input was non-zero:

3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9 (using position mode)
3,3,1105,-1,9,1101,0,0,12,4,12,99,1 (using immediate mode)
Here's a larger example:

3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99
The above example program uses an input instruction to ask for a single number. The program will then output 999 if the input value is below 8, output 1000 if the input value is equal to 8, or output 1001 if the input value is greater than 8.
```

This time, when the TEST diagnostic program runs its input instruction to get the ID of the system to test, provide it 5, the ID for the ship's thermal radiator controller. This diagnostic test suite only outputs one number, the diagnostic code.

What is the diagnostic code for system ID 5?


In [263]:
# Opcode 5 is jump-if-true: 
# if the first parameter is non-zero, 
# it sets the instruction pointer to the value from the second parameter. 
# Otherwise, it does nothing.

function op5(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 != 0
        return p2
    else
        return p + 3
    end
end

op5 (generic function with 1 method)

In [264]:
# Opcode 6 is jump-if-false: 
# if the first parameter is zero, 
# it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.

function op6(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 == 0
        return p2
    else
        return p + 3
    end
end

op6 (generic function with 1 method)

In [265]:
# Opcode 7 is less than: 
# if the first parameter is less than the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op7(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 < p2
        prog[pos(prog[pos(p+3)])] = 1
    else
        prog[pos(prog[pos(p+3)])] = 0
    end
    p + 4
end

op7 (generic function with 1 method)

In [266]:
# Opcode 8 is equals: 
# if the first parameter is equal to the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op8(operation,p,prog) 
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if p1 == p2
        prog[pos(prog[pos(p+3)])] = 1
    else
        prog[pos(prog[pos(p+3)])] = 0
    end
    p + 4
end

op8 (generic function with 1 method)

In [267]:
@enum Op plus=1 mul=2 stor=3 out=4 jnz=5 jez=6 lt=7 eq=8 halt=99
@enum Mode posi=0 imm=1

struct Operation 
   op::Op 
   mode1::Mode
   mode2::Mode
   mode3::Mode
end

In [268]:
function execute(op::Operation, p, prog, input)
    if op.op == plus
        return op1(op,p,prog)
    end
    if op.op == mul
        return op2(op,p,prog)
    end
    if op.op == stor
        return op3(op,p,prog,input)
    end
    if op.op == out
        return op4(op,p,prog)
    end
    if op.op == jnz
        return op5(op,p,prog)
    end
    if op.op == jez
        return op6(op,p,prog)
    end   
    if op.op == lt
        return op7(op,p,prog)
    end   
    if op.op == eq
        return op8(op,p,prog)
    end    
    
    error("panic executing unknown op")
end

execute (generic function with 4 methods)

In [269]:
function execute(prog,input)
    p = 0
    op = getOps(prog[pos(p)])
    #println("op: ", op)
    while op.op != halt
        #println(prog)
        p = execute(op,p,prog,input)   
        #println("instruction: ",p, " -> ",prog[pos(p)])
        op = getOps(prog[pos(p)])
    end 
    prog
end

execute (generic function with 4 methods)

In [273]:
execute([3,9,8,9,10,9,4,9,99,-1,8],8); # should output 1 if input == 8 (pos mode)
execute([3,9,8,9,10,9,4,9,99,-1,8],9); # should output 0 if input != 8 (pos mode)
execute([3,9,7,9,10,9,4,9,99,-1,8],1); # should output 1 if input < 8 (pos mode)
execute([3,9,7,9,10,9,4,9,99,-1,8],10); # should output 0 if input > 8 (pos mode)
execute([3,3,1108,-1,8,3,4,3,99],8); # should output 1 if input == 8 (imm mode)
execute([3,3,1108,-1,8,3,4,3,99],9); # should output 0 if input != 8 (imm mode)
execute([3,3,1107,-1,8,3,4,3,99],7); # should output 1 if input < 8 (imm mode)
execute([3,3,1107,-1,8,3,4,3,99],9); # should output 0 if input > 8 (imm mode)
execute([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9],0); # output 0 if the input was zero (pos mode)
execute([3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9],1); # output 1 if the input was non-zero (pos mode)
execute([3,3,1105,-1,9,1101,0,0,12,4,12,99,1],0); # output 0 if the input was zero (imm mode)
execute([3,3,1105,-1,9,1101,0,0,12,4,12,99,1],-1); # output 1 if the input was non-zero (imm mode)

output:1
output:0
output:1
output:0
output:1
output:0
output:1
output:0
output:0
output:1
output:0
output:1


In [277]:
# The above example program uses an input instruction to ask for a single number. 
# The program will then 
# output 999 if the input value is below 8,
# output 1000 if the input value is equal to 8, 
# or output 1001 if the input value is greater than 8.

execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,-8);

execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,8);

execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,16);

output:999
output:1000
output:1001


In [279]:
using DelimitedFiles
prog = readdlm("input.txt", '\t', Int, ',')
execute(prog,5);

# should output 3892695

output:3892695
